In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [2]:
# 周期映射: daily/weekly/monthly/3month -> yfinance interval
import time

INTERVAL_MAP = {'daily': '1d', 'weekly': '1wk', 'monthly': '1mo', '3month': '3mo'}

def data_prep(ticker, start="2010-01-01", end="2025-12-31", interval="daily", data=None, max_retries=3, retry_delay=5):
    """
    快速生成 data preparation（失败时自动重试）
    
    参数:
        ticker: 股票代码（当 data 为 None 时使用）
        start, end: 日期范围（当 data 为 None 时使用）
        interval: 周期 - 'daily'(日线), 'weekly'(周线), 'monthly'(月线), '3month'(三个月线)
        data: 可选，直接传入 OHLC DataFrame（需含 Open/High/Low/Close/Adj Close）
        max_retries: 下载失败时重试次数，默认 3
        retry_delay: 重试前等待秒数，默认 5
    
    返回:
        含 High-Low, True Range Percentage 列的 DataFrame
    """
    if data is None:
        iv = INTERVAL_MAP.get(interval, interval)  # 支持 '1d' 等直接传入
        last_err = None
        for attempt in range(max_retries):
            try:
                raw = yf.download(ticker, start=start, end=end, interval=iv, auto_adjust=False)
                if raw is None or (isinstance(raw, pd.DataFrame) and raw.empty):
                    raise ValueError(f"yfinance 未返回数据: {ticker}")
                df = raw[['Open', 'High', 'Low', 'Close', 'Adj Close']].copy()
                break
            except Exception as e:
                last_err = e
                if attempt < max_retries - 1:
                    time.sleep(retry_delay)
                else:
                    raise last_err
    else:
        df = data[['Open', 'High', 'Low', 'Close', 'Adj Close']].copy()
    
    df = df.sort_index(ascending=False)  # descending by date (newest first)
    
    adj = df['Adj Close']
    if isinstance(adj, pd.DataFrame):
        adj = adj.iloc[:, 0]
    # 计算 High-Low: High - Low
    df['High-Low'] = df['High'] - df['Low']
    # 计算  True Range Percentage: MAX(High-Low, |High-下一行Close|, |Low-下一行Close|) / Open
    prev_close = df['Close'].shift(-1)
    if isinstance(prev_close, pd.DataFrame):
        prev_close = prev_close.iloc[:, 0]
    high, low, open_ = df['High'], df['Low'], df['Open']
    if isinstance(high, pd.DataFrame):
        high, low, open_ = high.iloc[:, 0], low.iloc[:, 0], open_.iloc[:, 0]
    tr = np.maximum(high - low, np.maximum(np.abs(high - prev_close), np.abs(low - prev_close)))
    df['TR%'] = tr / open_ * 100

    return df

In [3]:
def ATRP(df, windows=None):
    """
    计算各周期 TR% 均值（Average True Range Percentage）。
    数据需按时间降序（最新在前），第1~N行对应最近N个交易日。

    参数:
        df: 含 TR% 列的 DataFrame
        windows: 可选，[(交易日数, 窗口名), ...]，默认 1周~20年（ATRP(prices, windows=[(5,'1周'), (20,'1月')])：自定义窗口）

    返回:
        DataFrame: 窗口、交易日数、TR%均值
    """
    if windows is None:
        windows = [
            (5, '1周'),
            (20, '1月'),
            (60, '1季'),
            (250, '1年'),
            (750, '3年'),
            (1250, '5年'),
            (2500, '10年'),
            (5000, '20年'),
        ]
    tr_pct = df['TR%'].dropna()
    if isinstance(tr_pct, pd.DataFrame):
        tr_pct = tr_pct.iloc[:, 0]
    return pd.DataFrame({
        '窗口': [name for _, name in windows],
        '交易日数': [n for n, _ in windows],
        'TR%均值': [tr_pct.iloc[:n].mean() if len(tr_pct) >= n else np.nan for n, _ in windows],
    })

In [ ]:
prices = data_prep("SPY", start="2000-01-01", end="2020-11-04", interval="daily")
prices.head()


In [ ]:
ATRP(prices)

In [4]:
# Asset universe: Asset, Class, Notes
assets_df = pd.DataFrame([
    # Government Bonds
    ("TLT", "Government Bonds", "20+ yr"),
    ("IEF", "Government Bonds", "7-10 yr"),
    ("IEI", "Government Bonds", "3-7 yr"),
    ("SHY", "Government Bonds", "1-3 yr"),
    # Corporate Bonds
    ("LQD", "Corporate Bonds", "Investment Grade"),
    ("VCIT", "Corporate Bonds", "Investment Grade"),
    ("JNK", "Corporate Bonds", "Junk"),
    ("HYG", "Corporate Bonds", "Junk"),
    # FX Major
    ("EURUSD", "FX Major", None),
    ("USDJPY", "FX Major", None),
    ("GBPUSD", "FX Major", None),
    ("AUDUSD", "FX Major", None),
    # FX EM
    ("USDZAR", "FX EM", None),
    ("USDBRL", "FX EM", None),
    ("USDTRY", "FX EM", None),
    # Equity Index
    ("SPY", "Equity Index", "S&P 500"),
    ("QQQ", "Equity Index", "Nasdaq 100"),
    ("DIA", "Equity Index", "Dow Jones Industrial"),
    ("IWV", "Equity Index", "Russell 3000"),
    ("IWM", "Equity Index", "Russell 2000"),
    # Equity - Mega Cap (FAANMG)
    ("META", "Equity - Mega Cap", "FAANMG"),
    ("AAPL", "Equity - Mega Cap", "FAANMG"),
    ("AMZN", "Equity - Mega Cap", "FAANMG"),
    ("NFLX", "Equity - Mega Cap", "FAANMG"),
    ("MSFT", "Equity - Mega Cap", "FAANMG"),
    ("GOOG", "Equity - Mega Cap", "FAANMG"),
    # Equity - Large Cap
    ("MAR", "Equity - Large Cap", "Discretionary/Industrial"),
    ("LVS", "Equity - Large Cap", "Discretionary/Industrial"),
    ("LEN", "Equity - Large Cap", "Discretionary/Industrial"),
    ("BBY", "Equity - Large Cap", "Discretionary/Industrial"),
    ("ODFL", "Equity - Large Cap", "Discretionary/Industrial"),
    ("LUV", "Equity - Large Cap", "Discretionary/Industrial"),
    ("PCAR", "Equity - Large Cap", "Discretionary/Industrial"),
    ("JCI", "Equity - Large Cap", "Discretionary/Industrial"),
    ("WMT", "Equity - Large Cap", "Discretionary/Industrial/Tech?"),
    ("HSY", "Equity - Large Cap", "F&B, Healthcare, Utilities"),
    ("CPB", "Equity - Large Cap", "F&B, Healthcare, Utilities"),
    ("STZ", "Equity - Large Cap", "F&B, Healthcare, Utilities"),
    ("MNST", "Equity - Large Cap", "F&B, Healthcare, Utilities"),
    ("BIIB", "Equity - Large Cap", "F&B, Healthcare, Utilities"),
    ("ALGN", "Equity - Large Cap", "F&B, Healthcare, Utilities"),
    ("XEL", "Equity - Large Cap", "F&B, Healthcare, Utilities"),
    ("PPL", "Equity - Large Cap", "F&B, Healthcare, Utilities"),
    ("SNDK", "Equity - Large Cap", "Semiconductor"),
    ("MU", "Equity - Large Cap", "Semiconductor"),
    ("TSM", "Equity - Large Cap", "Semiconductor"),
    ("PLTR", "Equity - Large Cap", "AI/Military"),
    # Equity - Mid Cap
    ("WING", "Equity - Mid Cap", None),
    ("FRPT", "Equity - Mid Cap", None),
    ("BILL", "Equity - Mid Cap", None),
    ("PEGA", "Equity - Mid Cap", None),
    ("OLED", "Equity - Mid Cap", None),
    ("XPO", "Equity - Mid Cap", None),
    ("FND", "Equity - Mid Cap", None),
    ("TREX", "Equity - Mid Cap", None),
    ("MOS", "Equity - Mid Cap", None),
    ("OLLI", "Equity - Mid Cap", None),
    ("DKS", "Equity - Mid Cap", None),
    ("VIRT", "Equity - Mid Cap", None),
    ("JBLU", "Equity - Mid Cap", None),
], columns=["Asset", "Class", "Notes"])

In [5]:
# 方法2：每个标的仅下载一次日线 + 行数映射 + 请求间延迟（降低 yfinance 限速）
import time

def to_yf_ticker(ticker):
    if ticker in ['EURUSD', 'USDJPY', 'GBPUSD', 'AUDUSD', 'USDZAR', 'USDBRL', 'USDTRY']:
        return f"{ticker}=X"
    return ticker

def _to_scalar(x):
    arr = np.asarray(x).ravel()
    return float(arr[0]) if len(arr) > 0 and np.isfinite(arr[0]) else np.nan

# 交易日数映射: 全样本=每日, 5=1周, 20=1月, 60=1季
ROW_MAP = [(None, 'Daily'), (5, 'Weekly'), (20, 'Monthly'), (60, '3Month')]
ATRP_COLS = [f"ATRP_{s}(%)" for _, s in ROW_MAP]

def compute_atrp_for_assets(assets_df, start="2010-01-01", end="2025-12-31", delay_sec=2, use_tqdm=True):
    """
    为 assets_df 中每个标的计算 ATRP（Daily/Weekly/Monthly/3Month），拼接至原表后返回。
    每个标的仅下载一次日线，按行数映射计算，请求间加延迟降低限速。
    """
    if use_tqdm:
        try:
            from tqdm import tqdm
            iter_assets = tqdm(assets_df['Asset'], desc="ATRP(单次下载+延迟)")
        except ImportError:
            iter_assets = assets_df['Asset']
    else:
        iter_assets = assets_df['Asset']
    atrp_data = []
    for asset in iter_assets:
        time.sleep(delay_sec)
        row = []
        try:
            df = data_prep(to_yf_ticker(asset), start=start, end=end, interval='daily')
            tr_pct = df['TR%'].dropna()
            if isinstance(tr_pct, pd.DataFrame):
                tr_pct = tr_pct.iloc[:, 0]
            for n, _ in ROW_MAP:
                if n is None:
                    val = _to_scalar(tr_pct.mean()) if len(tr_pct) > 0 else np.nan
                else:
                    val = _to_scalar(tr_pct.iloc[:n].mean()) if len(tr_pct) >= n else np.nan
                row.append(val)
        except Exception:
            row = [np.nan] * len(ROW_MAP)
        atrp_data.append(row)
    base = assets_df.drop(columns=ATRP_COLS, errors='ignore')
    return pd.concat([base, pd.DataFrame(atrp_data, columns=ATRP_COLS)], axis=1)

In [6]:
assets_df = compute_atrp_for_assets(assets_df, start="2025-01-01", end="2025-12-31", delay_sec=2)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [9]:
assets_df.to_csv("export/AverageTrueRangePercentage_per_asset.csv", index=False)

In [ ]:
# 按 Class 聚合：每个 class 的平均 ATRP
atrp_cols = ['ATRP_Daily(%)', 'ATRP_Weekly(%)', 'ATRP_Monthly(%)', 'ATRP_3Month(%)']
agg_cols = [c for c in atrp_cols if c in assets_df.columns]
if 'Class' not in assets_df.columns:
    raise ValueError("assets_df 缺少 Class 列")
if not agg_cols:
    raise ValueError("assets_df 缺少 ATRP 列，请先运行上方 compute_atrp_for_assets")
atrp_class_df = assets_df.groupby('Class')[agg_cols].mean().reset_index()

,Class,ATRP_Daily(%),ATRP_Weekly(%),ATRP_Monthly(%),ATRP_3Month(%)
0,Corporate Bonds,0.431152,0.220150,0.273786,0.327505
1,Equity - Large Cap,3.226140,1.697788,2.758501,3.184615
2,Equity - Mega Cap,2.646788,1.124632,2.027766,2.445793
3,Equity - Mid Cap,4.053997,2.207758,3.251192,3.833006
4,Equity Index,1.508208,0.574029,0.983294,1.339207
5,FX EM,1.018526,0.714913,0.765461,0.751776
6,FX Major,0.889280,0.607279,0.619194,0.675882
7,Government Bonds,0.470560,0.247227,0.304387,0.333539


In [10]:
atrp_class_df.to_csv("export/AverageTrueRangePercentage_per_class.csv", index=False)